# Entropic interactions, phase transitions
## Clothes pins on a line
- In the following instead of hard disks we use 1D (clothes) pins of size $2\sigma$ on a line $L$.
- We will see that the pins experience a strong force: Depletion Interaction (fifth force in nature)
    - Entropic interactions, Phase Transition
- $\pi(x_0, \dots, x_{N-1} = \begin{cases} 1 & \text{if legal}\\ 0 & \text{otherwise} \end{cases}$
- Legal conditions: $ \sigma < x_i < L - \sigma$ and $ | x_i - x_j | >= 2 \sigma$
- Algorithm: Similar to direct disks, create the configuration (step by step) and check for validity (and apply tabula rasa rule)
- Optimization: Create the configuration, sort the pins, check only for neighborhing pins.

In [ ]:
import random, timeit

def direct_pins(N, L, sigma):
    configuration = []
    while len(configuration) < N:
        configuration.append(random.uniform(sigma, L - sigma))
        for k in range(len(configuration) - 1): 
            if abs(configuration[-1] - configuration[k]) < 2.0 * sigma:
                configuration = []
                break
    return configuration

def direct_pins_improved(N,L,sigma):
    while True:
        configuration = [random.uniform(sigma, L - sigma) for k in range(N)]
        configuration.sort()
        min_dist = min(configuration[k + 1] - configuration[k] for k in range(N - 1)) 
        if min_dist > 2.0 * sigma:
            return configuration

# Larger N and L, so the faster algorithm is more prominent
# Timing a random algorithm is problematic
# The improved algorithm is better, when there is large N
# and low density so only few rejections and thus repetitions will be produced
N = 400
L = 10000.0
sigma = 0.1
%timeit direct_pins(N,L,sigma)
%timeit direct_pins_improved(N,L,sigma)

## The Asakura-Oosawa Interaction
- At higher density the probability of a pin being on a position x is not uniform
![Probability of a pin at postion x](images/plot-direct_pins_noreject.png)
- Two types of excluded regions:
    - Core of the pin (red)
    - Halo around the pin, (and halo at the boundaries) (white)
![Probability of a pin at postion x](images/plot-direct_pins-configuration.png)

- $\Rightarrow$ the available area of the line is dependent on the positions of the pins
- $\Rightarrow$ a configuration with a large available area has a higher chance of getting one more disk
- $\Rightarrow$ higher probability for closeby disks, and disks near a boundary
- System is homogeneous on long length scales $\Rightarrow$ liquid state
- Mathematic theorems exclude the possibilites of a phase transition in 1D
- ???In 2D/3D powerfull enough to see on large length scale, so a phase transition is possible???

## Asakura-Oosawa: Physical Systems
- 1D
    - Close-packed limit $\eta = 1$
    - Periodic boundary conditions $x_0$ is not known, altough all pins are all next to each other $\Rightarrow$ Long-range order
    - At $\eta < 1 \Rightarrow$ state like behaviour
- 2D
    - Close-packed density $\eta = \frac{\pi}{2 \sqrt{3}} \approx 0.907$
    - Closed-packing configuration: **Hexagonial**
    - At small $\eta \Rightarrow$ liquid like
    - At intermediate $\eta \Rightarrow$ Numerical simulation
    - $\Rightarrow$ local arrengements which resemble the close-packing configuration
- 3D similar to 2D

In [ ]:
# From exercises in Week2
import random, pylab

def markov_disks(L, sigma):
    sigma_sq = sigma ** 2
    delta = 0.1                                                                                                                                                                    
    a = random.choice(L)
    b = [a[0] + random.uniform(-delta, delta), a[1] + random.uniform(-delta, delta)]
    min_dist = min((b[0] - c[0]) ** 2 + (b[1] - c[1]) ** 2 for c in L if c != a)
    box_cond = min(b[0], b[1]) < sigma or max(b[0], b[1]) > 1.0 - sigma
    if not (box_cond or min_dist < 4.0 * sigma ** 2): 
        a[:] = b 
        #L[:] = L
    return L


def direct_disks_box(N, sigma):
    overlap = True
    while overlap == True:
        L = [(random.uniform(sigma, 1.0 - sigma), random.uniform(sigma, 1.0 - sigma))]
        for k in range(1, N):
            a = (random.uniform(sigma, 1.0 - sigma), random.uniform(sigma, 1.0 - sigma))
            min_dist_sq = min(((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) for b in L)
            if min_dist_sq < 4.0 * sigma ** 2:
                overlap = True
                break
            else:
                overlap = False
                L.append(a)
    return L


N = 4
sigma = 0.1197
L = [[0.25, 0.25], [0.75, 0.25], [0.25, 0.75], [0.75, 0.75]]
n_runs = 2000
histo_data = []
for run in range(n_runs):
    pos = markov_disks(L, sigma)
    for k in range(N):
        histo_data.append(pos[k][0])
pylab.hist(histo_data, bins=100, normed=True)
pylab.xlabel('x')
pylab.ylabel('frequency')
pylab.title('Direct sampling: x coordinate histogram (density eta=0.18)')
pylab.grid()
pylab.savefig('direct_disks_histo.png')
pylab.show()

# TODO show it also for direct_disks_box

## Rejection free algorithm
- Only the available area is considered for new pins
- $\Rightarrow$ cut out locked regions
- For $N$ pins: $L_{new} = (L - 2 N \sigma)$
- To retrieve positions of the pins back to the old $L$: $x_i = y_i + 2(i+1)\sigma$
- The algorithm is reversed, starting with $L-2N\sigma$ and inflating the area with pins
- Difficult to generalize for 2D or 3D

In [ ]:
import random

# N = 10
# L = 20.0
#sigma = 0.75
def direct_pins_noreject(N,L,sigma):
    y = [random.uniform(0.0, L - 2 * N * sigma) for k in range(N)]
    y.sort()
    return [y[i] + (2 * i + 1) * sigma for i in range(N)]

%timeit direct_pins(N,L,sigma)
%timeit direct_pins_improved(N,L,sigma)
%timeit direct_pins_noreject(N,L,sigma)

## Exact Solution
- Partion function of the system can be computed
- $Z(\eta) = \int_{\sigma}^{L-\sigma}dx_0 \dots dx_{N-1} \pi(x_0, \dots, x_{N-1}) = $ number of legal configurations for densitye $\eta$
- Sorting step is crutial $\rightarrow$ doing the same in the integral
- $Z(\eta) = N! \int_{\sigma}^{L-\sigma}dx_0 \dots dx_{N-1} \pi(x_0, \dots, x_{N-1}) \times \Theta(x_0, \dots, x_{N-1})$
- $\Theta(x_0, \dots, x_{N-1}) = \begin{cases}1 & x_0 < \dots x_{N-1}\\0 & otherwise\end{cases}$
- Deflation step: change variables $\rightarrow$ losing the ordering and thus $\pi()$
- $Z(\eta) = N! \int_{0}^{L-2N\sigma}dy_0 \dots dy_{N-1} \Theta(x_0, \dots, x_{N-1})$
- $= \int_{0}^{L-2N\sigma}dy_0 \dots dy_{N-1}$ ( $\Theta$ picks out one of the N factorial permutations of the integral)
- $\Rightarrow Z = \begin{cases}(L-2N\sigma)^N & L>2N\sigma\\ 0 & \text{otherwise}\end{cases}$
- $p_{\text{accept}} = \frac{\text{number of legal configurations for density }\eta}{\text{number of legal and illegal configurations}} = \frac{Z(\eta)}{Z(0)}$
- (Number of configuration wipe-outs for direct_pins $ \approx \frac{1}{p_{\text{accept}}}(\eta)$)
- Virial expansion: $\frac{V}{N} \frac{\partial \log Z}{\partial V} = \frac{L}{N} \frac{\partial \log Z}{\partial L}$ for $Z = (L-2N\sigma)^N$
- $\Rightarrow \frac{L}{L-2N\sigma} = \frac{1}{1-\eta}$ for $\eta = \frac{2\sigma N}{L}$ which is the pin density
- Infinite virial expansion: $\frac{1}{(1-\eta)}= 1 + \eta + \eta^2 + \eta^3 + \dots$
- This series converges for all $eta < 1 \Rightarrow$ no phase transition
- For $\eta = 1$ special case, long range order



- With $Z(\eta)$ we can compute the distribution of $\pi(x)$
- $\pi(x):$ probability of having a pin center at x
- Also the probability, given the pin center at x, to put other pins on its left and right
- add $k$ pins on left: $Z_{k,x-\sigma} = ((x-\sigma) - 2\sigma k)^k$
- add $N-k-1$ pins on right: $Z_{N-k-1,L-x-\sigma} = ((L-x-\sigma) - 2\sigma(N-k-1))^{N-k-1}$
- $\Rightarrow \pi(x) = \sum_{k=0}^{N-1} \frac{1}{Z_{N,L}} \binom{N-1}{k} Z_{k,x-\sigma} Z_{N-1-k,L-x-\sigma}$
- $\binom{N-1}{k}$ number of choises of picking k left pins out of the bucket of the $N-1$ pins that remain once we placed the first one at x



- $\pi(\sigma) = \frac{1}{1-\eta} \pi(L/2)$
- $rightarrow$ very large at high density (pins are strongly attracted to the walls)

In [ ]:
import pylab

def binomialCoeff(n, k):
    result = 1
    for i in range(1, k+1):
        result = result * (n-i+1) / i
    return result

def Z(N, L, sigma):
    freespace = L - 2.0 * N * sigma
    if freespace > 0.0:
        result = freespace ** N
    else:
        result = 0.0
    return result

def pi(x, N, L, sigma):
    tot = 0.
    for k in range(0, N):
        Z1 = Z(k, x - sigma, sigma)
        Z2 = Z(N - k - 1, L - x - sigma, sigma)
        tot += binomialCoeff( N - 1, k) * Z1 * Z2
    Ztotal = Z(N, L, sigma)
    return tot / Ztotal

L = 20.0
N = 10
sigma = 0.75
xr = pylab.linspace(0.0, L, 201)
yr = [pi(x, N, L, sigma) for x in xr]
pylab.plot(xr, yr, 'red', linewidth=2.0)
pylab.xlabel('$x$', fontsize=14)
pylab.ylabel('$\pi(x)$', fontsize=14)
pylab.title('Exact density of %i clothes-pins ($\sigma$=%s)\non a line of length L=%s' % (N, sigma, L))
#pylab.savefig('plot-direct_pins_density.png')
pylab.show()

## Periodic boundaries
- With periodic boundaries is $\pi()$ uniform?
- $\rightarrow$ line becomes circle
- Placing one pin anywhere splits the circle into a line again
- However the placement of the frist pin is uniform
- Therefore the distribution function remains uniform

In [ ]:
import random, pylab

N = 24
L = 30.0
sigma = 0.5
n_runs = 10000
data = []
for run in range(n_runs):
    Lprime = L - 2.0 * sigma
    y_sorted = [random.uniform(0, Lprime - 2.0 * (N - 1) * sigma) for k in range(N - 1)]
    y_sorted.sort()
    sample = [y_sorted[k] + (2.0 * k + 1.0) * sigma for k in range(N - 1)] + [L - sigma]
    shift = random.uniform(0, L)
    data += [(y + shift) % L for y in sample]
pylab.title('Density of %i clothes-pins ($\sigma$=%s) on a line of length L=%s' % (N, sigma, L))
pylab.xlabel('$x$', fontsize=14)
pylab.ylabel('$\pi(x)$', fontsize=14)
pylab.title('Density profile $\pi(x)$ for N=%i, $\sigma$=%.2f, L=%.1f' % (N, sigma, L))
pylab.hist(data, bins=100, normed=True)
#pylab.savefig('plot-pins_noreject_periodic-N%04i-L%.1f-density.png' % (N, L))
pylab.show()

- Oscilations close to the poles have vanished
- Correlations between pins remain
- $\pi(x,x') =$ probability of having a pint at postition x and at position x'
- $\pi(x,x') =$ Constant if no correlations

In [ ]:
import random, pylab

def dist(x1, x2, L):
    d_x = abs(x1 - x2) 
    return min(d_x, L - d_x)

N = 450
L = 500.0
sigma = 0.5
density = N * 2.0 * sigma / L
n_runs = 100
x_max = 30.0  # maximum of the histogram range
data, pair_corr = [], []
for run in range(n_runs):
    Lprime = L - 2.0 * sigma
    y_sorted = [random.uniform(0, Lprime - 2.0 * (N - 1.0) * sigma) for k in range(N - 1)]
    y_sorted.sort()
    sample = [y_sorted[k] + (2.0 * k + 1.0) * sigma for k in range(N - 1) ] + [L - sigma]
    pair_corr += [dist(sample[i], sample[j], L) for i in range(N) for j in range(i)]
histo, bins, patches = pylab.hist(pair_corr, bins=800, normed=True)
pylab.xlim(0.0, x_max)
pylab.title('Pair-correlation function $\pi(x,y)$\nN=%i, $\sigma$=%.2f, L=%.1f, density=%.2f' % (N, sigma, L, density))
pylab.xlabel('$|x-y|$', fontsize=14)
pylab.ylabel('$\pi(|x-y|)$', fontsize=14)
#pylab.savefig('plot-pins_noreject_periodic-N%04i-L%.1f-pair_corr.png' % (N, L))
pylab.show()
pylab.clf()
asymptotic_val = 1.0 / (L / 2.0)   # asymptotic value of the pair correlation function
pylab.semilogy(bins[:-1], [abs(y - asymptotic_val) for y in histo])
pylab.xlim(0.0, x_max)
pylab.title('Deviation of $\pi(x,y)$ from its asymptotic value\nN=%i, $\sigma$=%.2f, L=%.1f, density=%.2f' % (N, sigma, L, density))
pylab.xlabel('$|x-y|$', fontsize=14)
pylab.ylabel('$|\pi(|x-y|)-\pi_\mathrm{asympt}|$', fontsize=14)
#pylab.savefig('plot-pins_noreject_periodic-N%04i-L%.1f-pair_corr_deviation.png' % (N, L))
pylab.show()

- Same oscilations as with boundaries
- Pin x acts as a pole
- $\Rightarrow \pi_{N,L}(x,x') = \pi_{N-1,L-2\sigma}(|x-x'|-\sigma)$


- From the logarithmic scale:
    - Correlation function decays exponentially $\exp(x/\xi)$
    - $\xi$ represents correlation lenghts between pins in the system
    - at $\eta = 0.8 \Rightarrow \xi \approx 2.4$
    - at $\eta = 0.9 \Rightarrow \xi \approx 6.4$
    - at $\eta = 0.95 \Rightarrow \xi \approx 26.2$
    - $\Rightarrow \xi$ diverges as $\eta \rightarrow 1$
- TODO sum up???

## Homework

In [ ]:
import random, math, pylab, os, cmath, numpy

# Implements periodic boundaries
def dist(x,y):
    d_x = abs(x[0] - y[0]) % 1.0
    d_x = min(d_x, 1.0 - d_x)
    d_y = abs(x[1] - y[1]) % 1.0
    d_y = min(d_y, 1.0 - d_y)
    return  math.sqrt(d_x**2 + d_y**2)

# Dinstance vector
def delx_dely(x, y):
    d_x = (x[0] - y[0]) % 1.0
    if d_x > 0.5: d_x -= 1.0
    d_y = (x[1] - y[1]) % 1.0
    if d_y > 0.5: d_y -= 1.0
    return d_x, d_y

def markov_disks(L, sigma):
    sigma_sq = sigma ** 2
    delta = 0.1                                                                                                                                                                    
    a = random.choice(L)
    b = [(a[0] + random.uniform(-delta, delta)) % 1., (a[1] + random.uniform(-delta, delta)) % 1.]
    if not (min(dist(b,c) for c in L if c != a) < 2*sigma): 
        a[:] = b 
    return L

def snapshot(pos):
    pylab.subplots_adjust(left=0.10, right=0.90, top=0.90, bottom=0.10)
    pylab.gcf().set_size_inches(6, 6)
    pylab.axis([0, 1, 0, 1])
    pylab.setp(pylab.gca(), xticks=[0, 1], yticks=[0, 1], aspect = 'equal')
    for (x, y) in pos:
        for ix in range(-1,2):
            for iy in range(-1,2):
                circle = pylab.Circle((x+ix, y+iy), radius=sigma, fc='r')
                pylab.gca().add_patch(circle)
    pylab.show()
    pylab.close()
    
def get_L(filename, N_sqrt):
    L = []
    if os.path.isfile(filename):
        f = open(filename, 'r')
        for line in f:
            a, b = line.split()
            L.append([float(a), float(b)])
        f.close()
    else:
        for k in range(3):
            delxy = sigma *1.3
            L = [[sigma + 1/N_sqrt * i, sigma + 1/N_sqrt * j] for i in range(N_sqrt) for j in range(N_sqrt)]
    return L

def Psi_6(L, sigma):
    sum_vector = 0j
    for i in range(N):
        vector  = 0j
        n_neighbor = 0
        for j in range(N):
            if dist(L[i], L[j]) < 2.8 * sigma and i != j:
                n_neighbor += 1
                dx, dy = delx_dely(L[j], L[i])
                angle = cmath.phase(complex(dx, dy))
                vector += cmath.exp(6.0j * angle)
        if n_neighbor > 0:
            vector /= n_neighbor
        sum_vector += vector
    return sum_vector / float(N)

def psi_6(phi):
    return 1/len(phi) * sum(cmath.exp(6j * p) for p in phi)

eta = 0.72
N_sqrt = 8
N = N_sqrt**2
sigma = math.sqrt(eta / (N * math.pi))
L = get_L('disk_configurations.txt', N_sqrt)
n_runs = 10000
psi = []
pos = []
snapshot(L)
for eta in numpy.arange(0.72, 0.2, -0.02):
    sigma = math.sqrt(eta / (N * math.pi))
    psi_eta = 0
    for run in range(n_runs):
        pos = markov_disks(L, sigma)
        if run % 100 == 0:
            psi_eta += (abs(Psi_6(pos,sigma)))
    psi.append(psi_eta/n_runs*100)
snapshot(pos)

pylab.plot(numpy.arange(0.72, 0.2, -0.02), psi, 'r--', linewidth=1.5, label="Psi_6")
pylab.xlabel('Eta')
pylab.ylabel('Value of Psi_6')
pylab.legend()
pylab.show()


In [ ]:
import math
def psi_6(phi):
    return sum((cmath.exp(6j * p) for p in phi)) / complex(len(phi),0)
phi_1 = [i*60 for i in range(6)]
phi_2 = [i*60+30 for i in range(6)]
phi_3 = [i*60+15 for i in range(6)]
print(psi_6(phi_1))
print(psi_6(phi_2))
print(psi_6(phi_3))
print(psi_6(phi_1) + psi_6(phi_2))
# TODO why is this wrong?